# P2. Les vues SQL

## Définition d'une vue SQL  

Une vue n'est rien de plus qu'une instruction SQLite qui est stockée dans la base de données avec un nom associé. Il s'agit en fait d'une composition d'une table sous la forme d'une requête SQLite prédéfinie.

Une vue peut contenir toutes les lignes d'une table ou des lignes sélectionnées d'une ou plusieurs tables. Une vue peut être créée à partir d'une ou plusieurs tables qui dépendent de la requête SQLite écrite pour créer une vue.

## Code utilitaire

In [1]:
import os
import sqlite3
import pandas as pd   

In [2]:
from pathlib import Path

In [3]:
# Ouvre connection vers SQLite db
chemin_bdd = Path('db') / 'chinook.db'
conn = sqlite3.connect(chemin_bdd)

In [4]:
# Fonction permettant d'exécuter un requête SQL sur une BDD définie par sa connexion conn
def executer_requete(requete_sql, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(requete_sql)
        conn.commit()
    except sqlite3.Error as e:
        print("Erreur lors de l'execution de la requête")
        print(e)
        return
    cursor.close()

## Example de vue SQL: v_albums

La requête suivante permet d'obtenir une liste (simplifiée) des albums / artistes correspondants:  

In [5]:
# Requete sur les tables albums & artists
requete = """
    SELECT AlbumId,
           Title,
           a.Name AS Artist
    FROM albums
    JOIN artists a on albums.ArtistId = a.ArtistId
    ORDER BY Artist;
"""

albums = pd.read_sql_query(requete, conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


Le code ci-dessous permet de créer une vue correspondant à la requête ci-dessous :    
Référence vue sous SQLite : https://www.sqlite.org/lang_createview.html

In [6]:
# creation vue albums avec artistes
creation_vue_albums = """
    CREATE VIEW v_albums (
        AlbumId,
        Title,
        Artist)
    AS
        SELECT AlbumId,
               Title,
               a.Name AS Artist
        FROM albums
        JOIN artists a on albums.ArtistId = a.ArtistId
        ORDER BY Artist, Title;
"""

executer_requete(creation_vue_albums, conn)

Erreur lors de l'execution de la requête
table v_albums already exists


Une fois la vue créé, on peut la réutiliser dans des requêtes SQL comme une table:  

In [7]:
# Utilisation de la vue v_albums
albums = pd.read_sql_query("SELECT * FROM v_albums LIMIT 10;", conn)
albums.head(5)

,AlbumId,Title,Artist,Time,Price
0,1,For Those About To Rock We Salute You,AC/DC,144024900000,9.90
1,2,Balls to the Wall,Accept,20553720000,0.99
2,3,Restless and Wild,Accept,51485280000,2.97
3,4,Let There Be Rock,AC/DC,147195540000,7.92
4,5,Big Ones,Aerosmith,264702540000,14.85


In [8]:
# Utilisation de la vue v_albums avec une clause WHERE
albums = pd.read_sql_query("SELECT * FROM v_albums WHERE Artist = 'U2';", conn)
albums

,AlbumId,Title,Artist,Time,Price
0,232,Achtung Baby,U2,199709340000,11.88
1,233,All That You Can't Leave Behind,U2,177876240000,10.89
2,234,B-Sides 1980-1990,U2,215693100000,14.85
3,235,How To Dismantle An Atomic Bomb,U2,176961060000,10.89
4,236,Pop,U2,216754320000,11.88
5,237,Rattle And Hum,U2,260887560000,16.83
6,238,The Best Of 1980-1990,U2,236154960000,13.86
7,239,War,U2,151794180000,9.90
8,240,Zooropa,U2,181395060000,9.90
9,255,Instant Karma: The Amnesty International Campa...,U2,308093160000,22.77


In [9]:
# Utilisation de la vue v_albums avec une autre jointure
requete = """
    SELECT a.AlbumId,
           a.Title,
           a.Artist,
           count(t.TrackId) as Tracks
    FROM v_albums a
    JOIN tracks t ON a.AlbumId = t.AlbumId
    GROUP BY 1, 2, 3
    ORDER BY 3, 2;
"""
albums = pd.read_sql_query(requete, conn)
albums.head()

,AlbumId,Title,Artist,Tracks
0,1,For Those About To Rock We Salute You,AC/DC,10
1,4,Let There Be Rock,AC/DC,8
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1
3,267,Worlds,Aaron Goldberg,1
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,2


In [10]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS v_albums;", conn)

### Question: Avantages & Inconvénients des vues SQL

Quelles sont les avantages et inconvénients des vues SQL ?  

*Avantages :*

- Gain de temps en cas de nombreuses utilisations d'une information
- Simplification de la complexité des requêtes

*Inconvénients :*

- Ralentissement des performances des requêtes
- A actualiser en cas de modification des données dans la BDD existante

---

## Exercices de création / mise en oeuvre de vues

### Vue 1: caractéristiques complètes des Albums  
A partir de l'exemple de vue ci-dessus, créer et exploiter une nouvelle vue v_albums reprenant pour chaque album:  
- AlbumId
- Title
- Artist
- nb de pistes
- Durée totale de l'album (en minutes)
- Prix de l'album

In [11]:
creation_v_albums = """
    CREATE VIEW v_albums (
        AlbumId,
        Title,
        Artist,
        Time,
        Price)
    AS
        SELECT albums.AlbumId,
        albums.Title,
        artists.Name,
        SUM(Milliseconds)*60000,
        SUM(tracks.UnitPrice)
        FROM albums
        JOIN artists ON albums.ArtistId = artists.ArtistId
        JOIN tracks ON albums.AlbumId = tracks.AlbumId
        GROUP BY albums.AlbumId
"""

executer_requete(creation_v_albums, conn)

In [12]:
albums2 = pd.read_sql_query("SELECT * FROM v_albums;", conn)
albums2

,AlbumId,Title,Artist,Time,Price
0,1,For Those About To Rock We Salute You,AC/DC,144024900000,9.90
1,2,Balls to the Wall,Accept,20553720000,0.99
2,3,Restless and Wild,Accept,51485280000,2.97
3,4,Let There Be Rock,AC/DC,147195540000,7.92
4,5,Big Ones,Aerosmith,264702540000,14.85
...,...,...,...,...,...
342,343,Respighi:Pines of Rome,Eugene Ormandy,17204460000,0.99
343,344,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet,8352000000,0.99
344,345,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",3998340000,0.99
345,346,Mozart: Chamber Music,Nash Ensemble,13279860000,0.99


In [13]:
executer_requete("DROP VIEW IF EXISTS v_tracks;", conn)

In [14]:
creation_v_tracks = """
    CREATE VIEW v_tracks (
        TrackId,
        Name,
        Genre,
        Album,
        Artist,
        Composer,
        Time,
        MediaType
        )
    AS
        SELECT tracks.TrackId,
        tracks.Name,
        genres.Name,
        albums.Title,
        artists.Name,
        tracks.Composer,
        (tracks.Milliseconds)*1000,
        media_types.Name
        FROM tracks
        JOIN albums ON tracks.AlbumId = albums.AlbumId
        JOIN artists ON albums.ArtistId = artists.ArtistId
        JOIN genres ON tracks.GenreId = genres.GenreId
        JOIN media_types ON tracks.MediaTypeId = media_types.MediaTypeId
        """

executer_requete(creation_v_tracks, conn)

In [15]:
tracks = pd.read_sql_query("SELECT * FROM v_tracks;", conn)
tracks

,TrackId,Name,Genre,Album,Artist,Composer,Time,MediaType
0,1,For Those About To Rock (We Salute You),Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",343719000,MPEG audio file
1,6,Put The Finger On You,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",205662000,MPEG audio file
2,7,Let's Get It Up,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",233926000,MPEG audio file
3,8,Inject The Venom,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",210834000,MPEG audio file
4,9,Snowballed,Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",203102000,MPEG audio file
...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,Classical,Respighi:Pines of Rome,Eugene Ormandy,None,286741000,Protected AAC audio file
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Classical,Schubert: The Late String Quartets & String Qu...,Emerson String Quartet,Franz Schubert,139200000,Protected AAC audio file
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",Classical,Monteverdi: L'Orfeo,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon...",Claudio Monteverdi,66639000,Protected AAC audio file
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Classical,Mozart: Chamber Music,Nash Ensemble,Wolfgang Amadeus Mozart,221331000,Protected AAC audio file


### Vue 3: Vue augmentée invoices
A. Créer une nouvelle vue v_invoices à partir de la table invoices ajoutant 3 nouveaux attributs basés sur l'attribut `InvoiceDate`:  
- semaine de la transaction
- mois de la transaction
- année de la transaction

In [35]:
executer_requete("DROP VIEW IF EXISTS v_invoices;", conn)

In [36]:
creation_v_invoices = """
    CREATE VIEW v_invoices (
        InvoiceId,
        CustomerId,
        InvoiceWeek,
        InvoiceMonth,
        InvoiceYear,
        BillingAddress,
        BillingCity,
        BillingState,
        BillingCountry,
        BillingPostalCode,
        Total)
    AS
        SELECT InvoiceId,
            CustomerId,
            STRFTIME("%W", InvoiceDate),
            STRFTIME("%M", InvoiceDate),
            STRFTIME("%Y", InvoiceDate),
            BillingAddress,
            BillingCity,
            BillingState,
            BillingCountry,
            BillingPostalCode,
            Total
        FROM invoices
    """

executer_requete(creation_v_invoices, conn)

In [39]:
v_invoices = pd.read_sql_query("SELECT * FROM v_invoices;", conn)
v_invoices

,InvoiceId,CustomerId,InvoiceWeek,InvoiceMonth,InvoiceYear,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,00,00,2009,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,00,00,2009,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,00,00,2009,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,01,00,2009,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,01,00,2009,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...,...,...
407,408,25,48,00,2013,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,409,29,48,00,2013,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,410,35,49,00,2013,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,411,44,49,00,2013,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


B. Exploiter la vue v_invoices ainsi crée pour déterminer les ventes annuelles par pays.

In [65]:
query3B = """
    SELECT DISTINCT BillingCountry AS "Pays", nine.Ventes AS "Ventes 2009", ten.Ventes AS "Ventes 2010", eleven.Ventes AS "Ventes 2011", twelve.Ventes AS "Ventes 2012", thirteen.Ventes AS "Vente 2013"
    FROM v_invoices
    
    LEFT JOIN
    (SELECT BillingCountry AS "Pays", SUM(Total) AS "Ventes"
    FROM v_invoices
    WHERE InvoiceYear = "2009"
    GROUP BY BillingCountry) AS nine
    ON v_invoices.BillingCountry = nine.Pays
    
    LEFT JOIN 
    (SELECT BillingCountry AS "Pays", SUM(Total) AS "Ventes"
    FROM v_invoices 
    WHERE InvoiceYear = "2010"
    GROUP BY BillingCountry) AS ten
    ON v_invoices.BillingCountry = ten.Pays
    
    LEFT JOIN
    (SELECT BillingCountry AS "Pays", SUM(Total) AS "Ventes"
    FROM v_invoices 
    WHERE InvoiceYear = "2011"
    GROUP BY BillingCountry) AS eleven
    ON v_invoices.BillingCountry = eleven.Pays
    
    LEFT JOIN
    (SELECT BillingCountry AS "Pays", SUM(Total) AS "Ventes"
    FROM v_invoices 
    WHERE InvoiceYear = "2012"
    GROUP BY BillingCountry) AS twelve
    ON v_invoices.BillingCountry = twelve.Pays
    
    LEFT JOIN
    (SELECT BillingCountry AS "Pays", SUM(Total) AS "Ventes"
    FROM v_invoices 
    WHERE InvoiceYear = "2013"
    GROUP BY BillingCountry) AS thirteen
    ON v_invoices.BillingCountry = thirteen.Pays
    
    ORDER BY "Pays"
"""

In [66]:
results = pd.read_sql_query(query3B, conn)
results

,Pays,Ventes 2009,Ventes 2010,Ventes 2011,Ventes 2012,Vente 2013
0,Argentina,NaN,11.88,0.99,NaN,24.75
1,Australia,11.88,0.99,1.98,22.77,NaN
2,Austria,1.98,27.77,NaN,11.88,0.99
3,Belgium,6.93,NaN,24.75,NaN,5.94
4,Brazil,37.62,41.60,19.80,53.46,37.62
5,Canada,57.42,76.26,55.44,42.57,72.27
6,Chile,15.84,17.91,5.94,6.93,NaN
7,Czech Republic,10.89,9.90,12.87,19.83,36.75
8,Denmark,5.94,6.93,NaN,15.84,8.91
9,Finland,8.91,NaN,15.88,0.99,15.84


### Vue 4: A vous de jouer
Créer et exploiter une nouvelle vue de votre choix  

La vue suivante concerne la liste des clients pour laquelle nous avons également également ajouté l'employé référent. Cette vue nous permettra ensuite de connaître le nom de clients par employés sans avoir besoin de réaliser de jointures.

In [63]:
creation_v_customers = """
CREATE VIEW v_customers (
        CustomerId,
        Customer,
        Company,
        Address,
        City,
        State,
        Country,
        PostalCode,
        Phone,
        Fax,
        Email,
        EmployeeId,
        Employee)
    AS
		SELECT CustomerId,
			customers.firstname || ' ' || customers.lastname,
			customers.Company,
			customers.Address,
			customers.City,
			customers.State,
			customers.Country,
			customers.PostalCode,
			customers.Phone,
			customers.Fax,
			customers.Email,
            EmployeeId,
			employees.firstname || ' ' || employees.lastname
		FROM customers
		LEFT JOIN employees ON customers.SupportRepId = employees.EmployeeId
"""

executer_requete(creation_v_customers, conn)

In [64]:
v_customers = pd.read_sql_query("SELECT * FROM v_customers;", conn)
v_customers

,CustomerId,Customer,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,EmployeeId,Employee
0,1,Luís Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3,Jane Peacock
1,2,Leonie Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5,Steve Johnson
2,3,François Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3,Jane Peacock
3,4,Bjørn Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4,Margaret Park
4,5,František Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4,Margaret Park
5,6,Helena Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5,Steve Johnson
6,7,Astrid Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,None,astrid.gruber@apple.at,5,Steve Johnson
7,8,Daan Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,None,daan_peeters@apple.be,4,Margaret Park
8,9,Kara Nielsen,None,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,+453 3331 9991,None,kara.nielsen@jubii.dk,4,Margaret Park
9,10,Eduardo Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4,Margaret Park


In [ ]:
query4 = """
SELECT Employee, count(*) As "Nb clients"
FROM v_customers
GROUP BY EmployeeId"